In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats

diabetes = pd.read_csv('data/diabetes_data.csv')
#diabetes.info()

dupl_columns = list(diabetes.columns)
mask = diabetes.duplicated(subset=dupl_columns)
sber_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {sber_duplicates.shape[0]}')
sber_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {sber_dedupped.shape[0]}')


#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

information_diabetes = sber_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')

#ПРОПУСКИ

cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
information_diabetes[cols] = sber_dedupped[cols].replace({'0':np.nan, 0:np.nan})
information_diabetes.isnull().mean()
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')




n = information_diabetes.shape[0] #число строк в таблице
thresh = n*0.7
combine_information_diabetes = information_diabetes.dropna(thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_information_diabetes.shape[1] #число признаков после удаления столбцов
combine_information_diabetes = combine_information_diabetes.dropna(axis=0,thresh=m-2)
print(information_diabetes.shape[0])
print(f'Результирующее число записей ДО: {combine_information_diabetes.shape[0]}')

combine_information_diabetes = combine_information_diabetes.replace(np.nan, combine_information_diabetes.median())
combine_information_diabetes['SkinThickness'].mean()
data = combine_information_diabetes.copy()
data.shape[0]

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(data, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(data, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768
Gender: 100.0% одинаковых значений
Результирующее число признаков: 9
Результирующее число признаков: 9
768
Результирующее число записей ДО: 761
Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761
